In [6]:
import pandas as pd
import numpy as np

# Cargo la base limpia con la que hemos venido trabajando
df = pd.read_csv("df_cleaned.csv")

# Aseguro tipos básicos que voy a usar
df["price"] = df["price"].astype(float)
df["accommodates"] = df["accommodates"].astype(int)

# Precio por huésped como indicador de valor relativo
df["price_per_guest"] = df["price"] / df["accommodates"]

# Reconstruyo el borough a partir de las dummies de neighbourhood_group_cleansed
borough_cols = [
    "neighbourhood_group_cleansed:Bronx",
    "neighbourhood_group_cleansed:Brooklyn",
    "neighbourhood_group_cleansed:Manhattan",
    "neighbourhood_group_cleansed:Queens",
    "neighbourhood_group_cleansed:Staten Island"
]

df["borough_seg"] = (
    df[borough_cols]
      .idxmax(axis=1)          # columna que tiene el 1
      .str.split(":", n=1)
      .str[1]                  # me quedo con Bronx, Brooklyn, etc.
)

# Reconstruyo el tipo de habitación a partir de las dummies de room_type
room_cols = [
    "room_type:Entire home/apt",
    "room_type:Hotel room",
    "room_type:Private room",
    "room_type:Shared room"
]

df["room_type_seg"] = (
    df[room_cols]
      .idxmax(axis=1)
      .str.split(":", n=1)
      .str[1]
)

# Reviso rápido que las nuevas columnas tengan sentido
print(df[["price", "accommodates", "price_per_guest", "borough_seg", "room_type_seg"]].head())

# Guardo esta base intermedia para el modelo 3
df.to_csv("df_modelo3_segmentos_base.csv", index=False)


   price  accommodates  price_per_guest borough_seg    room_type_seg
0   66.0             1        66.000000      Queens     Private room
1   76.0             1        76.000000   Manhattan     Private room
2   97.0             6        16.166667      Queens  Entire home/apt
3   60.0             1        60.000000    Brooklyn     Private room
4  425.0             6        70.833333    Brooklyn  Entire home/apt


In [7]:
# Cargo la base intermedia generada en el commit 1
df = pd.read_csv("df_modelo3_segmentos_base.csv")

# Defino el segmento comparable: mismo borough + mismo tipo de habitación
segment_cols = ["borough_seg", "room_type_seg"]

# Para cada segmento calculo:
# - percentil 25 y 75 de price_per_guest (rango razonable de precio por huésped)
# - mediana de amenities_count (mínimo razonable de amenidades)
segment_stats = (
    df.groupby(segment_cols)
      .agg(
          p25_ppg=("price_per_guest", lambda x: np.percentile(x, 25)),
          p75_ppg=("price_per_guest", lambda x: np.percentile(x, 75)),
          med_amenities=("amenities_count", "median")
      )
      .reset_index()
)

# Uno estas estadísticas al dataframe principal
df = df.merge(segment_stats, on=segment_cols, how="left")

# Regla de recomendación:
# - price_per_guest entre p25 y p75 del segmento
# - amenities_count >= med_amenities del segmento
cond_precio = (df["price_per_guest"] >= df["p25_ppg"]) & (df["price_per_guest"] <= df["p75_ppg"])
cond_amenities = df["amenities_count"] >= df["med_amenities"]

df["recommended"] = np.where(cond_precio & cond_amenities, 1, 0)

# Reviso que la etiqueta no quede extremadamente desbalanceada
print("Distribución de la etiqueta recommended:")
print(df["recommended"].value_counts(dropna=False))
print(df["recommended"].value_counts(normalize=True).round(3))

# Guardo la base final para el modelo de clasificación
df.to_csv("df_modelo3_clasificacion.csv", index=False)

Distribución de la etiqueta recommended:
recommended
0    15283
1     5544
Name: count, dtype: int64
recommended
0    0.734
1    0.266
Name: proportion, dtype: float64


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Cargo la base ya con la etiqueta recommended
df = pd.read_csv("df_modelo3_clasificacion.csv")

# Variable objetivo: recommended (0 = no recomendada, 1 = recomendada)
y = df["recommended"].astype(int)

print("Distribución de la clase objetivo:")
print(y.value_counts(dropna=False))
print(y.value_counts(normalize=True).round(3))

# Defino qué columnas NO quiero usar como features
# (target, textos, columnas de ayuda para construir la etiqueta, etc.)
cols_excluir = [
    "recommended",          # target
    "amenities",            # lista de amenities en texto
    "bathrooms_text",       # texto descriptivo del baño
    "calendar_last_scraped",
    "host_since",
    "host_response_time",
    "price_range",
    "borough_seg",          # ya se usó para definir la regla de negocio
    "room_type_seg",        # igual
    "p25_ppg",
    "p75_ppg",
    "med_amenities"
]

# Me quedo solo con las columnas que sí van a entrar al modelo
# (tanto numéricas como dummies que ya vienen en el dataset)
cols_features = [c for c in df.columns if c not in cols_excluir]

X = df[cols_features].copy()

print("\nNúmero de variables explicativas seleccionadas:", X.shape[1])

# Split train / test estratificado para respetar la proporción de clases
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Tamaños de los conjuntos:")
print("X_train:", X_train.shape)
print("X_test :", X_test.shape)

# Escalado estándar de todas las features (los dummies pasan de 0/1 a valores centrados, lo cual está bien para la red)
scaler_clf = StandardScaler()
scaler_clf.fit(X_train)

X_train_scaled = scaler_clf.transform(X_train)
X_test_scaled = scaler_clf.transform(X_test)

# Paso a float32 por compatibilidad con TensorFlow
X_train_scaled = X_train_scaled.astype("float32")
X_test_scaled = X_test_scaled.astype("float32")

y_train = y_train.values.astype("float32")
y_test = y_test.values.astype("float32")

print("\nEjemplo de fila escalada:")
print(X_train_scaled[0][:10])


Distribución de la clase objetivo:
recommended
0    15283
1     5544
Name: count, dtype: int64
recommended
0    0.734
1    0.266
Name: proportion, dtype: float64

Número de variables explicativas seleccionadas: 100
Tamaños de los conjuntos:
X_train: (16661, 100)
X_test : (4166, 100)

Ejemplo de fila escalada:
[-3.9586241  -2.802569    0.12077124  1.4168756  -0.43168676 -0.3549193
 -0.37398288 -0.54305923 -0.13032001  0.14498988]
